# QKD BB84
## Summary

```
for i = 1..n {
  // Alice
  with a new random bit b {
    with a random polarisation p selected from R or D {

             0°                     R = + (rectilinear)
             |                      D = X (diagonal)
             |
      --------------- 90°
             |
             |

      if (p == R)
        if (b == 0) Alice -----Photon at 0° -----> Bob
        if (b == 1) Alice -----Photon in 90° -----> Bob
      if (p == D)
        if (b == 0) Alice -----Photon at 45° -----> Bob
        if (b == 1) Alice -----Photon at 135° -----> Bob
    }
    Record b,p

    // Eve
    Possibly try to read the photon

    // Bob
    with a random polarisation p selected from R or D {
      measure photon p as r
      record p,r
    }
  }
}

Bob -----polarisation used for each bit (public)-----> Alice
Alice -----Which polarisations were correct-----> Bob

// Alice and Bob
Remove the bits that were read with the wrong polarisation

Alice -----Selected keystring bits (public)-----> Bob
Bob -----match/mismatch (public)-----> Alice

if match, both Alice and Bob remove the test key bits from their keystring, resulting in a shared key.

```

In [1]:
import math, random

In [ ]:
NumberOfPhotons = 1000
Peavesdrop = 0.005

AliceBases = []
AliceKey = []
BobBases = []
BobKey = []

for i in range(NumberOfPhotons):
  # Alice
  AliceKey.append(random.randint(0,1))
  AliceBases.append(random.randint(0,1))
  if   AliceBases[-1] == 0 and AliceKey[-1] == 0: p = 0
  elif AliceBases[-1] == 0 and AliceKey[-1] == 1: p = 90
  elif AliceBases[-1] == 1 and AliceKey[-1] == 0: p = 45
  elif AliceBases[-1] == 1 and AliceKey[-1] == 1: p = 135

  # Eve
  if random.random() < Peavesdrop: # Chance Eve tries to read this photon
    # Assume Eve knows what polarisations we are using
    # When she tries to read a photon, its polarisation will be set
    #if random.randint(0,1) == 0: p = -45
    #else: p = 90
    pass

  
